・アカウント作成
→ csv作成ツールを参照
→ A01,02，またはB01,02を作成

・エイリアス更新
→ forAdmin2つを参照
→ B03を作成

・アカウント削除
→ forAdmin2つを参照
→ D01~03を作成

In [ ]:
##################################
### 必ず最初に実行する！！！！ ###
##################################

# 必要なライブラリのインストールおよびインポート
!pip install openpyxl

from google.colab import files
import pandas as pd
import numpy as np
import csv
import random
import string

# 数値型かどうかを確認する関数
def is_numeric(value):
  return isinstance(value, (int, float, complex))

# 削除されたデータをD01~03に書き込む関数
def write_deleted_data(old_data, new_data, csv_files):
  # 古いデータを一つずつループして削除されたか確認する
  for row_pointer_old in range(len(old_data)):
    # 数値型なら2023とかNanとかの関係ないものなのでスキップ
    if is_numeric(old_data[row_pointer_old][0]):
      continue
    found = False # フラグの用意
    # 新しいデータを一つずつ照らし合わせる
    for row_pointer_new in range(len(new_data)):
      if is_numeric(new_data[row_pointer_new][0]):
        continue
      # 一致する（= 削除されてない）ならフラグを立てて次のループへ
      if old_data[row_pointer_old][0] == new_data[row_pointer_new][0]:
        found = True
        break
    # 内側ループが終わった理由が最後までやり切ったことなら，削除対象データなのでcsvファイルに追加
    if not found:
      # アカウントの話かエイリアスの話かで分岐
      data_address = old_data[row_pointer_old][3] if old_data == accounts_old else str(old_data[row_pointer_old][0]) + '@rm2c.ise.ritsumei.ac.jp'
      data_name = "rm2c-admin." + data_address[:data_address.find('@')]
      # CSVファイルを追記モードで開く
      with open(csv_files[0], 'a', newline='', encoding='cp932') as csvfile:
        csvwriter = csv.writer(csvfile)
        csvwriter.writerows([[data_name, '', '', '1', '0']])
      with open(csv_files[1], 'a', newline='', encoding='cp932') as csvfile:
        csvwriter = csv.writer(csvfile)
        csvwriter.writerows([['2', data_address, data_name]])
      with open(csv_files[2], 'a', newline='', encoding='cp932') as csvfile:
        csvwriter = csv.writer(csvfile)
        csvwriter.writerows([['3', data_name, '', '', '', '', '', '', '']])

# エイリアス名とそこに紐づけられたメルアドの辞書を作成する関数
def create_alias_dict(alias_data, alias_dict):
  account_name = ''
  mail_address_list = ''
  for row_pointer in range(len(alias_data)):
    if is_numeric(alias_data[row_pointer][0]):
      mail_address_list += '\n' + str(alias_data[row_pointer][5])
    else:
      if account_name:
        alias_dict[account_name] = mail_address_list
      account_name = 'rm2c-admin.' + alias_data[row_pointer][0]
      mail_address_list = alias_data[row_pointer][5]
  if account_name:
    alias_dict[account_name] = mail_address_list

# 指定範囲内の文字をランダムに一文字抽出
def rand_between(lower_bound, upper_bound):
  return random.randint(lower_bound, upper_bound)

# 指定された文字列からランダムに一文字抽出
def random_char_picker(source):
  location = rand_between(0, len(source) - 1)
  return source[location]

# 文字列をシャッフル
def shuffle_string(source):
  char_list = list(source)
  random.shuffle(char_list)
  return ''.join(char_list)

# ランダムにパスワードを生成する関数
def random_password_generator():
  #########################################################################
  ############# パスワードの生成方式については機密事項のためここでは省略 ###########
  password = 'abcde'
  #########################################################################
  
  return password

In [ ]:
############################################
### エイリアス更新およびアカウント削除用 ###
############################################

# ファイルのアップロード
print('古いforAdominファイルをアップロードしてください')
table_file_old = files.upload()
print('新しいforAdminファイルをアップロードしてください')
table_file_new = files.upload()

# アップロードしたファイル名を取得
table_old = list(table_file_old.keys())[0]
table_new = list(table_file_new.keys())[0]

# pandasを使用してエクセルファイルを読み込み、二次元配列に変換
accounts_table_old = pd.read_excel(table_old, sheet_name='アカウント')
accounts_table_new = pd.read_excel(table_new, sheet_name='アカウント')
aliasis_table_old = pd.read_excel(table_old, sheet_name='エイリアス')
aliasis_table_new = pd.read_excel(table_new, sheet_name='エイリアス')

# データフレームを二次元配列（リストのリスト）に変換
accounts_old = accounts_table_old.values.tolist()
accounts_new = accounts_table_new.values.tolist()
aliasis_old = aliasis_table_old.values.tolist()
aliasis_new = aliasis_table_new.values.tolist()

# 表の行数を取得
num_accounts_rows_old = len(accounts_old)
num_accounts_rows_new = len(accounts_new)
num_aliasis_rows_old = len(aliasis_old)
num_aliasis_rows_new = len(aliasis_new)

# 作成するファイル名
b03 = 'B03_alias_skeleton.csv'
d01 = 'D01_alias_skeleton.csv'
d02 = 'D02_mailaddress_skeleton.csv'
d03 = 'D03_aliasaccount_skeleton.csv'

# ファイル名と初期データの辞書
file_initial_data = {
  b03: [['アカウント名', '備考', '転送先メールアドレス', '転送後削除', '迷惑メールを転送しない']],
  d01: [['アカウント名', '備考', '転送先メールアドレス', '転送後削除', '迷惑メールを転送しない']],
  d02: [['処理区分', '独自ドメインメールアドレス', '配信先アカウント名']],
  d03: [['処理区分', 'アカウント名', 'パスワード', '備考', 'メールボックス容量', 'メールボックス使用量', 'パスワード変更設定権限', '転送設定設定権限', '自動返信設定権限', 'メールアドレス（セレクトドメイン）設定権限']]
}

# CSVファイルを作成および初期化
for filename, data in file_initial_data.items():
  with open(filename, 'w', newline='', encoding='cp932') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerows(data)

# エイリアスの削除されたデータを処理
write_deleted_data(aliasis_old, aliasis_new, [d01, d02, d03])
# アカウントの削除されたデータを処理
write_deleted_data(accounts_old, accounts_new, [d01, d02, d03])

# 古いエイリアス辞書の作成
alias_dict_old = {}
create_alias_dict(aliasis_old, alias_dict_old)
# 新しいエイリアス辞書の作成
alias_dict_new = {}
create_alias_dict(aliasis_new, alias_dict_new)

# 新しい辞書のキーを一つずつ参照して、oldに存在しないキーやoldと値が異なるキーを抽出する
key_diffs = []
for key, value in alias_dict_new.items():
  old_value = alias_dict_old.get(key)
  if not (is_numeric(old_value) and is_numeric(value)):
    if key not in alias_dict_old or alias_dict_old[key] != value:
      key_diffs.append(key)

# 変更があったエイリアスをB03に追加する
for key in key_diffs:
  if is_numeric(alias_dict_new[key]):
    data = [[key, '', '', '1', '0']]
  else:
    data = [[key, '', alias_dict_new[key], '1', '0']]
  with open(b03, 'a', newline='', encoding='cp932') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerows(data)

# 作成したファイルをダウンロードする
for filename, data in file_initial_data.items():
    files.download(filename)

古いforAdominファイルをアップロードしてください


Saving 20240410_forAdmin.xlsx to 20240410_forAdmin.xlsx
新しいforAdminファイルをアップロードしてください


Saving 20240508_forAdmin.xlsx to 20240508_forAdmin.xlsx


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
########################################
### アカウントおよびエイリアス作成用 ###
########################################

# 作成するファイル名
a01 = 'A01_account_skeleton.csv'
a02 = 'A02_mailaddress_skeleton.csv'
b01 = 'B01_aliasaccount_skeleton.csv'
b02 = 'B02_mailaddress_skeleton.csv'

# ファイル名と初期データの辞書
file_initial_data = {
  a01: [['処理区分', 'アカウント名', 'パスワード', '備考', 'メールボックス容量', 'メールボックス使用量', 'パスワード変更設定権限', '転送設定設定権限', '自動返信設定権限', 'メールアドレス（セレクトドメイン）設定権限']],
  a02: [['処理区分', '独自ドメインメールアドレス', '配信先アカウント名']],
  b01: [['処理区分', 'アカウント名', 'パスワード', '備考', 'メールボックス容量', 'メールボックス使用量', 'パスワード変更設定権限', '転送設定設定権限', '自動返信設定権限', 'メールアドレス（セレクトドメイン）設定権限']],
  b02: [['処理区分', '独自ドメインメールアドレス', '配信先アカウント名']]
}

# CSVファイルを作成および初期化
for filename, data in file_initial_data.items():
  with open(filename, 'w', newline='', encoding='cp932') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerows(data)

while True:
  user_input = input('作成するアカウント名を入力してください．（例：hir-koba, rm2czoom，など）終了する場合は"0"を入力してください： ')
  if user_input.lower() == '0':
    break
  mail_address = user_input + '@rm2c.ise.ritsumei.ac.jp'
  account_name = 'rm2c-admin.' + user_input
  data02 = [['1', mail_address, account_name]]
  user_input = input("これはアカウントですか？エイリアスですか？（アカウント：0，エイリアス：1）: ")
  if user_input.lower() == '0':
    remarks = input("備考欄の氏名を入力して下さい．（例：“小林洋輝”，“YUN Jonghee”，など）： ")
    user_input = input("これは学生ですか？教員ですか？（学生：0，教員：1）: ")
    if user_input.lower() == '0':
      data01 = [['1', account_name, random_password_generator(), remarks, '300', '1', '2', '2', '2', '0']]
    elif user_input.lower() == '1':
      data01 = [['1', account_name, random_password_generator(), remarks, '1024', '1', '2', '2', '2', '0']]
    else:
      print("無効な入力です．入力し直してください．")
    with open(a01, 'a', newline='', encoding='cp932') as csvfile:
      csvwriter = csv.writer(csvfile)
      csvwriter.writerows(data01)
    with open(a02, 'a', newline='', encoding='cp932') as csvfile:
      csvwriter = csv.writer(csvfile)
      csvwriter.writerows(data02)
  elif user_input.lower() == '1':
    user_input = input('備考欄を入力して下さい．（例：“rm2c共有zoomアカウント”，“rm2cのVive使用者”，など）： ')
    remarks = 'Alias: ' + user_input
    data01 = [['1', account_name, random_password_generator(), remarks, '10', '1', '0', '0', '0', '0']]
    with open(b01, 'a', newline='', encoding='cp932') as csvfile:
      csvwriter = csv.writer(csvfile)
      csvwriter.writerows(data01)
    with open(b02, 'a', newline='', encoding='cp932') as csvfile:
      csvwriter = csv.writer(csvfile)
      csvwriter.writerows(data02)
  else:
    print("無効な入力です．入力し直してください．")


# 作成したファイルをダウンロードする
for filename, data in file_initial_data.items():
    files.download(filename)

作成するアカウント名を入力してください．（例：hir-koba, rm2czoom，など）終了する場合は"0"を入力してください： ma-mrt
これはアカウントですか？エイリアスですか？（アカウント：0，エイリアス：1）: 0
備考欄の氏名を入力して下さい．（例：“小林洋輝”，“YUN Jonghee”，など）： 森田磨里絵
これは学生ですか？教員ですか？（学生：0，教員：1）: 1
作成するアカウント名を入力してください．（例：hir-koba, rm2czoom，など）終了する場合は"0"を入力してください： ob2023
これはアカウントですか？エイリアスですか？（アカウント：0，エイリアス：1）: 1
備考欄を入力して下さい．（例：“rm2c共有zoomアカウント”，“rm2cのVive使用者”，など）： 2023年度卒OB
作成するアカウント名を入力してください．（例：hir-koba, rm2czoom，など）終了する場合は"0"を入力してください： staff2024
これはアカウントですか？エイリアスですか？（アカウント：0，エイリアス：1）: 1
備考欄を入力して下さい．（例：“rm2c共有zoomアカウント”，“rm2cのVive使用者”，など）： 2024年度B3配属用
作成するアカウント名を入力してください．（例：hir-koba, rm2czoom，など）終了する場合は"0"を入力してください： guidance2024
これはアカウントですか？エイリアスですか？（アカウント：0，エイリアス：1）: 1
備考欄を入力して下さい．（例：“rm2c共有zoomアカウント”，“rm2cのVive使用者”，など）： 2024年度B3配属用
作成するアカウント名を入力してください．（例：hir-koba, rm2czoom，など）終了する場合は"0"を入力してください： rm2czoom
これはアカウントですか？エイリアスですか？（アカウント：0，エイリアス：1）: 1
備考欄を入力して下さい．（例：“rm2c共有zoomアカウント”，“rm2cのVive使用者”，など）： RM2C共有zoomアカウント
作成するアカウント名を入力してください．（例：hir-koba, rm2czoom，など）終了する場合は"0"を入力してください： 0


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>